In [1]:
import torch
import torch.nn as nn
import torchvision.models as models

In [ ]:
model_vgg = models.vgg16(pretrained=True)

print(model_vgg)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [2]:
class FCN16(nn.Module):
    def __init__(self,model,hidden =64,kernel_size=3,padding = 1, num_classes = 21):
        super(FCN16,self).__init__()
        
        self.block1 = nn.Sequential(*list(model.features)[:5])
        self.block2 = nn.Sequential(*list(model.features)[5:10])
        self.block3 = nn.Sequential(*list(model.features)[10:17])
        self.block4 = nn.Sequential(*list(model.features)[17:24])
        self.block5 = nn.Sequential(*list(model.features)[24:31])
        
        model.classifier[0] = nn.Conv2d(512,4096,7)
        model.classifier[3] = nn.Conv2d(4096,4096,1)
        model.classifier[6] = nn.Conv2d(4096,num_classes,1)
        
        self.fc6 = nn.Sequential(*list(model.classifier)[0:3])
        self.fc7 = nn.Sequential(*list(model.classifier)[3:6])
        self.block_score = model.classifier[6]
        
        self.score_pool4 = nn.Conv2d(8*hidden,num_classes, kernel_size=1)
        
        self.upscore2 = nn.ConvTranspose2d(num_classes,num_classes,4,stride=2,bias=False)
        self.upscore16 = nn.ConvTranspose2d(num_classes,num_classes,32,stride = 16, bias=False)
        
    def forward(self,x):
        pred1 = self.block1(x)
        pred2 = self.block2(pred1)
        pred3 = self.block3(pred2)
        pred4 = self.block4(pred3)
        pred5 = self.block5(pred4)
        
        pred6 = self.fc6(pred5)
        pred7 = self.fc7(pred6)
        score = self.block_score(pred7)
        upscore2 = self.upscore2(score)
        
        pred4_1 = self.score_pool4(pred4)
        
        upscore16 = self.upscore16(upscore2+pred4_1)
        return upscore16

In [3]:
model = models.vgg16(pretrained=True)

fcn_model = FCN16(model)

print(fcn_model)

FCN16(
  (block1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (block3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
    (6): MaxPool